#Imports

INSTALL MORE RECENT LIGHTGBM VERSION

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM

In [ ]:
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

In [1]:
import lightgbm as lgbm

In [2]:
print(lgbm.__version__) # check version to be sure.

3.3.2.99


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
#import lightgbm # this imports lightgbm version 2.2.3, which does not support some pretty nice features.
import matplotlib.pyplot as plt
from math import log10


#Loading and checking data

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('drive/MyDrive/training_set_VU_DM.csv')#, nrows=100000)

In [4]:
df['price_usd'].replace(to_replace=0.0, value=1, inplace=True) # some prices are 0.0, which is weird. replace with 0.1 for the log function
df['price_usd'] = df['price_usd'].apply(log10)

df['visitor_hist_adr_usd'].replace(to_replace=0.0, value=1, inplace=True) # some prices are 0.0, which is weird.
df['visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].apply(log10)

In [5]:
### feature which highlights difference between the visitors historical data and the hotel cost
df['usd_diff'] = np.abs(df.visitor_hist_adr_usd.values - df.price_usd.values)
df['star_diff'] = np.abs(df.visitor_hist_starrating.values - df.prop_starrating.values)

#Making Training and evaluation data for the model

Adding mean property price as a feature to dataset

In [6]:
def get_from_dict(x, _dict):
    '''There might be a better way to do this'''
    return _dict[x]


### mean and std of 'price_usd' feature per prop_id ###
same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['price_usd'].mean().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'price_usd'+'_'+'avg'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['price_usd'].std().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'price_usd'+'_'+'std'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

'''
### mean and std of 'prop_starrating' feature ###
same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['prop_starrating'].mean().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'prop_starrating'+'_'+'avg'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['prop_starrating'].std().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'prop_starrating'+'_'+'std'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

### mean and std of 'prop_review_score' feature ###
same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['prop_review_score'].mean().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'prop_review_score'+'_'+'avg'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['prop_review_score'].std().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'prop_review_score'+'_'+'std'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

### mean and std of 'prop_log_historical_price' feature ###
same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['prop_log_historical_price'].mean().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'prop_log_historical_price'+'_'+'avg'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)

same_vals_df = pd.DataFrame()
_dict = df.groupby('prop_id')['prop_log_historical_price'].std().to_dict() # A dict with the operation applied to each group of the id 
same_vals_df['same_val_'+'prop_id'+'_'+'prop_log_historical_price'+'_'+'std'] = df['prop_id'].apply(get_from_dict, _dict=_dict)
df = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)
'''

"\n### mean and std of 'prop_starrating' feature ###\nsame_vals_df = pd.DataFrame()\n_dict = df.groupby('prop_id')['prop_starrating'].mean().to_dict() # A dict with the operation applied to each group of the id \nsame_vals_df['same_val_'+'prop_id'+'_'+'prop_starrating'+'_'+'avg'] = df['prop_id'].apply(get_from_dict, _dict=_dict)\ndf = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)\n\nsame_vals_df = pd.DataFrame()\n_dict = df.groupby('prop_id')['prop_starrating'].std().to_dict() # A dict with the operation applied to each group of the id \nsame_vals_df['same_val_'+'prop_id'+'_'+'prop_starrating'+'_'+'std'] = df['prop_id'].apply(get_from_dict, _dict=_dict)\ndf = pd.concat([df, same_vals_df.set_index(df.index)], axis=1)\n\n### mean and std of 'prop_review_score' feature ###\nsame_vals_df = pd.DataFrame()\n_dict = df.groupby('prop_id')['prop_review_score'].mean().to_dict() # A dict with the operation applied to each group of the id \nsame_vals_df['same_val_'+'prop_id'+'_'+'prop_

for test set

In [7]:
### so i want to fill the competition NaN values with 0s
### all other variables i will set NaNs to -10
comp_columnns = [comp_col for comp_col in df.columns if comp_col.startswith('comp')]
df[comp_columnns] = df[comp_columnns].fillna(0)
df = df.fillna(-10)

for test set!

In [18]:
comp_columnns = [comp_col for comp_col in df_test.columns if comp_col.startswith('comp')]
df_test[comp_columnns] = df_test[comp_columnns].fillna(0)
df_test = df_test.fillna(-10)
df_test = df_test.drop(['date_time', 'site_id', 'random_bool', 'srch_id', 'prop_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd'], axis=1, inplace=False)

In [9]:
del same_vals_df

CV part


In [8]:
idx=dict()
idx[0],idx[1],idx[2],idx[3],idx[4]=np.array_split(df['srch_id'].unique(),5)

In [32]:
def val_model(boosting='dart',num_leaves=255,min_data_in_leaf=1,min_sum_hessian_in_leaf=100, n_estimators=10, num_iterations=10, learning_rate=0.03, early_stopping_rounds=65,max_depth=-1):
  score=0
  for i in range(5):  
    df_train=df.loc[~df['srch_id'].isin(idx[i]),:]
    df_eval=df.loc[df['srch_id'].isin(idx[i]),:]
    qids_train = df_train.groupby('srch_id')['srch_id'].count().to_numpy()
    y_train = 5 * df_train['booking_bool'].values + df_train['click_bool'].values # booking_bool weighs for 5, clicking bool for 1
    X_train = df_train.drop(['date_time', 'random_bool', 'click_bool', 'booking_bool', 'position', 'site_id', 'gross_bookings_usd', 'srch_id', 'prop_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd'], axis=1) # 

    qids_eval = df_eval.groupby('srch_id')['srch_id'].count().to_numpy()
    y_eval = 5 * df_eval['booking_bool'].values + df_eval['click_bool'].values
    X_eval = df_eval.drop(['date_time', 'random_bool', 'click_bool', 'booking_bool', 'position', 'site_id', 'gross_bookings_usd', 'srch_id', 'prop_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd'], axis=1) # 
    model = lgbm.LGBMRanker(objective='lambdarank', metric='ndcg', boosting='dart',
                          num_leaves=num_leaves, min_data_in_leaf=min_data_in_leaf, min_sum_hessian_in_leaf=min_sum_hessian_in_leaf, n_estimators=n_estimators,max_depth=max_depth,
                          num_iterations=num_iterations, learning_rate=learning_rate, early_stopping_rounds=early_stopping_rounds, device_type='gpu', seed=42)
    del df_train, df_eval
    model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_eval, y_eval)],
        eval_group=[qids_eval],
        eval_at=[5],
    )
    del qids_eval,qids_train,X_eval,X_train,y_eval,y_train
    score+=model.best_score_['valid_0']['ndcg@5']
    del model
  
  return(score)


In [ ]:
val_model(num_leaves=10)

In [ ]:
results=pd.DataFrame(columns=['num_leaves','max_depth','score'])
for num_leaves in [10,20]:
  for max_depth in [-1,4]:
    score=val_model(num_leaves=num_leaves,max_depth=max_depth)
    results=results.append({'num_leaves':num_leaves,'max_depth':max_depth,'score':score}, ignore_index=True)

In [34]:
print(results)

   num_leaves  max_depth     score
0        10.0       -1.0  1.746999
1        10.0        4.0  1.744392
2        20.0       -1.0  1.776137
3        20.0        4.0  1.750639
